In [38]:
# default_exp datasets.genetic_file.__init__

# Genetic file formats

> API details.

In [1]:
#hide
# %load_ext autoreload
# %autoreload 2

from nbdev.showdoc import *
import pandas as pd
from corradin_ovp_utils.catalog import test_data_catalog, conf_test_data_catalog
from fastcore.test import ExceptionExpected

In [2]:
#export
from corradin_ovp_utils.datasets.schemas import SingleFilePathSchema, CaseControlFilePathSchema
from typing import Any, Dict, List, Optional, Literal, Union, Protocol
from pydantic import BaseModel, validator
from pathlib import Path, PosixPath
from fastcore.basics import basic_repr
import numpy as np
import pandas as pd
import abc

In [5]:
#export
class GeneticFileFormat(BaseModel):
    # sample_ids: Optional[str]
    sample_file: Optional[str]
    sample_ids: Optional[np.ndarray]
    file_path: Union[SingleFilePathSchema, CaseControlFilePathSchema] #Union[str, Path, Dict[int, Union[str, Path]]]
    __repr__ = basic_repr("file_path")
    
    @validator('file_path')
    def name_must_contain_space(cls, file_path):
        if not file_path.split_by_chromosome and not file_path.chrom_num:
            raise ValueError("Must specify `chrom_num` in the data catalog if `file_path` is not a template")
        return file_path

    def get_resolved_file_path(self, chrom=None):
        return self.file_path.get_full_file_path(chrom=chrom)
    
    def is_chrom_template(self):
        return self.file_path.split_by_chromosome
    
    class Config:
        arbitrary_types_allowed= True
        
class GeneticFileFormatInterfaceAPI(metaclass=abc.ABCMeta):
    @classmethod
    def __subclasshook__(cls, subclass):
        return (hasattr(subclass, 'get_geno_each_sample') or NotImplemented) #\
#                 and 
#                 callable(subclass.get_geno_each_sample) #and 
#                 hasattr(subclass, 'extract_text') and 
#                 callable(subclass.extract_text) or 
#                 NotImplemented)
    @abc.abstractmethod
    def get_geno_each_sample(rsid_dict: Dict[int, str],\
                             batch_size:int,\
                             excluded_sample_ids:List[str], **kwargs):
        raise NotImplementedError
    

def row_vectorize(f):
    @wraps(f)
    def wrapped_f(X, **kwargs):
        rows = X.reshape(-1, X.shape[-1])
        return np.reshape([f(row, **kwargs) for row in rows],
                          X.shape[:-1] + (-1,))
    return wrapped_f


def triplicate_converter(sample_df, *, genotype_df, sample_id, high_lim=.9, low_lim=0.3, NA_val = "NA"):
        """
        takes in allele_list in gen triplicate format and
        return new_list in letter format
        """
        
        
        rsid_genotype_df = genotype_df.loc[sample_df.index]
        #all_geno = SimpleNamespace(**test_genfile_format.get_genotypes_df().loc[rsid])
        
        geno_1, geno_2, geno_3 = [sample_df.astype(float)[col] for col in sample_df]
        AA_cond = (geno_1 >=high_lim) & (geno_2 < low_lim) & (geno_3 < low_lim)
        AB_cond = (geno_2 >= high_lim) & (geno_1 < low_lim) & (geno_3 < low_lim)
        BB_cond = (geno_3 >= high_lim) & (geno_1 < low_lim) & (geno_2 < low_lim)
        geno_df =  np.select([AA_cond, AB_cond, BB_cond],
                 [rsid_genotype_df["AA"], rsid_genotype_df["AB"], rsid_genotype_df["BB"]],
                 default = NA_val)
        geno_df = pd.DataFrame(geno_df, index = sample_df.index, columns = [sample_id])
        geno_df.columns.name= "sample_id"
        return geno_df


# def get_possible_geno_combinations(allele_1:str, allele_2:str):
#     geno_df["homo_ref"] = geno_df.loc[:,[self.ref_col]] *2
#     make_het_geno_func = lambda row: ''.join(sorted([row[self.ref_col],
#                                    row[self.alt_col]]))
#     geno_df["het"] = geno_df[[self.ref_col, self.alt_col]].apply(make_het_geno_func, axis=1)
#     geno_df["homo_alt"] = geno_df.loc[:,self.alt_col] *2


def get_possible_geno_combinations(geno_df, allele_1_col:str, allele_2_col:str):
    geno_df_allele_cols = geno_df[[allele_1_col, allele_2_col]].copy()
    geno_df_allele_cols.columns = geno_df_allele_cols.columns.get_level_values(0)
    geno_df_allele_cols["AA"] = geno_df_allele_cols.loc[:,[allele_1_col, allele_1_col]].apply("".join, axis=1)
    make_het_geno_func = lambda row: ''.join(sorted([row[allele_1_col],
                                   row[allele_2_col]]))
    geno_df_allele_cols["AB"] = geno_df_allele_cols[[allele_1_col, allele_2_col]].apply(make_het_geno_func, axis=1)
    geno_df_allele_cols["BB"] = geno_df_allele_cols.loc[:,[allele_2_col, allele_2_col]].apply("".join, axis=1)
    return geno_df_allele_cols

def get_geno_one_snp(row,  high_lim=0.9, low_lim=0.3, NA_val = np.nan):
    geno_1, geno_2, geno_3 = row
    homo_ref_cond = (geno_1 >=high_lim) & (geno_2 < low_lim) & (geno_3 < low_lim)
    het_cond = (geno_2 >= high_lim) & (geno_1 < low_lim) & (geno_3 < low_lim)
    homo_alt_cond = (geno_3 >= high_lim) & (geno_1 < low_lim) & (geno_2 < low_lim)
    geno_df =  np.select([homo_ref_cond, het_cond, homo_alt_cond],
             [0., 1., 2.],
             default = NA_val)
    #geno_df = pd.DataFrame(geno_df, index = sample_df.index, columns = [sample_id])
    #geno_df.columns.name= "sample_id"
    return geno_df